# 1.ライブラリの読み込み
上から順に
* グラフを書くためにmatplotlib
* データの整理のためにpandas
* 数学的な計算のためにnumpy
* 時系列データを扱うためにdatetime
* matplotlibでdatetimeのフォーマットを変えるためにmdates

In [272]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.dates as mdates
from scipy import signal

# 2.センサーデータの読みこみと確認
ファイルの読み込みに使用した変数名の命名則については以下の通りである．

例　セッション1のAさんのセンサデータ →　data1_a
　　セッション1のAさんのスケーリングデータ　→　data1_a_label_50

In [273]:
%time
#sensor_dataの読み込み

# data1_a = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-121953632-ags.csv', encoding="shift-jis")
# data1_b = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-122231707-ags.csv', encoding="shift-jis")
# data1_c = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-121953885-ags.csv', encoding="shift-jis")
# data1_d = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-121954011-ags.csv', encoding="shift-jis")
data2_a = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-145759193-ags.csv', encoding="shift-jis")
# data2_b = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-145733362-ags.csv', encoding="shift-jis")
# data2_c = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-150018208-ags.csv', encoding="shift-jis")
# data2_d = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-145806707-ags.csv', encoding="shift-jis")

#emotion_labelの読み込み

#data1_a_label_50 = pd.read_csv('emotion_label/A/emotion_A_20201111_1_50.csv')
#data1_a_label_60 = pd.read_csv('emotion_label/A/emotion_A_20201111_1_60.csv')
#data1_a_label_70 = pd.read_csv('emotion_label/A/emotion_A_20201111_1_70.csv')
#data1_a_label_80 = pd.read_csv('emotion_label/A/emotion_A_20201111_1_80.csv')

data2_a_label_50 = pd.read_csv('emotion_label/session2/A/emotion_A_20201111_2_50.csv')
data2_a_label_60 = pd.read_csv('emotion_label/session2/A/emotion_A_20201111_2_60.csv')
data2_a_label_70 = pd.read_csv('emotion_label/session2/A/emotion_A_20201111_2_70.csv')
data2_a_label_80 = pd.read_csv('emotion_label/session2/A/emotion_A_20201111_2_80.csv')

Wall time: 0 ns


## 2.1 TSND151加速度，角速度センサデータとスケーリングデータの中身
今回のセンサデータの列はセンサの種類，時間，加速度のx，y，z，角速度x，y，zで構成されており，加速度の単位は0.1mG，角速度の単位は0.01dpsである．

スケーリングデータの列は，レコーディング動画の経過時間，感情のレベル値，発話されたユーザ名となっている．emotion_levelの感情値については今回は以下のように考えた．

-4〜-1→negative  0→neutral  １〜4→positive

In [274]:
data2_a_label_50

,start_time,finish_time,emotion_level
0,3:26,3:30,0
1,3:27,3:31,0
2,3:39,3:43,-2
3,3:40,3:44,-2
4,3:41,3:45,-2
...,...,...,...
540,18:06,18:10,-2
541,18:07,18:11,-2
542,18:08,18:12,-2
543,18:09,18:13,-2


In [275]:
#欠損値削除
def dropna(label):
    a = label.dropna(how='all', axis=1)
    return a

#data1_a_label_50 = dropna(data1_a_label_50)
#data1_a_label_60 = dropna(data1_a_label_60)
#data1_a_label_70 = dropna(data1_a_label_70)
#data1_a_label_80 = dropna(data1_a_label_80)

data2_a_label_50 = dropna(data2_a_label_50)
data2_a_label_60 = dropna(data2_a_label_60)
data2_a_label_70 = dropna(data2_a_label_70)
data2_a_label_80 = dropna(data2_a_label_80)

In [276]:
#欠損値削除
def dropna(label):
    a = label.dropna(how='all', axis=0)
    return a

#data1_a_label_50 = dropna(data1_a_label_50)
#data1_a_label_60 = dropna(data1_a_label_60)
#data1_a_label_70 = dropna(data1_a_label_70)
#data1_a_label_80 = dropna(data1_a_label_80)

data2_a_label_50 = dropna(data2_a_label_50)
data2_a_label_60 = dropna(data2_a_label_60)
data2_a_label_70 = dropna(data2_a_label_70)
data2_a_label_80 = dropna(data2_a_label_80)

In [277]:
data2_a_label_50

,start_time,finish_time,emotion_level
0,3:26,3:30,0
1,3:27,3:31,0
2,3:39,3:43,-2
3,3:40,3:44,-2
4,3:41,3:45,-2
...,...,...,...
540,18:06,18:10,-2
541,18:07,18:11,-2
542,18:08,18:12,-2
543,18:09,18:13,-2


In [278]:
#生データの確認
display(data2_a)
display(data2_a_label_50)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z
0,ags,14:57:59.245,9801,898,-258,340,1,913
1,ags,14:57:59.250,9911,592,-290,291,44,559
2,ags,14:57:59.255,9996,431,-268,218,-188,279
3,ags,14:57:59.260,10150,453,-280,114,-84,-75
4,ags,14:57:59.265,10292,263,-122,-43,-267,-415
...,...,...,...,...,...,...,...,...
469692,ags,15:37:07.780,10184,356,313,-318,860,-20
469693,ags,15:37:07.785,10216,282,230,-190,1006,-117
469694,ags,15:37:07.790,10087,336,339,-68,1116,-208
469695,ags,15:37:07.795,10284,273,331,23,1134,-287


,start_time,finish_time,emotion_level
0,3:26,3:30,0
1,3:27,3:31,0
2,3:39,3:43,-2
3,3:40,3:44,-2
4,3:41,3:45,-2
...,...,...,...
540,18:06,18:10,-2
541,18:07,18:11,-2
542,18:08,18:12,-2
543,18:09,18:13,-2


In [279]:
#合成加速度と合成角速度の算出
def norm_order(sensor_data):
    sensor_data["合成加速度"] = np.sqrt(sensor_data["加速度X"]*sensor_data["加速度X"] + sensor_data["加速度Y"]*sensor_data["加速度Y"] + sensor_data["加速度Z"]*sensor_data["加速度Z"])
    sensor_data["合成角速度"] = np.sqrt(sensor_data["角速度X"]*sensor_data["角速度X"] + sensor_data["角速度Y"]*sensor_data["角速度Y"] + sensor_data["角速度Z"]*sensor_data["角速度Z"])
    
# norm_order(data1_a)
# norm_order(data1_b)
# norm_order(data1_c)
# norm_order(data1_d)
norm_order(data2_a)
# norm_order(data2_b)
# norm_order(data2_c)
# norm_order(data2_d)

In [280]:
data2_a

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,14:57:59.245,9801,898,-258,340,1,913,9845.433916,974.253560
1,ags,14:57:59.250,9911,592,-290,291,44,559,9932.899124,631.742036
2,ags,14:57:59.255,9996,431,-268,218,-188,279,10008.876111,400.885270
3,ags,14:57:59.260,10150,453,-280,114,-84,-75,10163.961285,160.240444
4,ags,14:57:59.265,10292,263,-122,-43,-267,-415,10296.082605,495.341296
...,...,...,...,...,...,...,...,...,...,...
469692,ags,15:37:07.780,10184,356,313,-318,860,-20,10195.026287,917.128126
469693,ags,15:37:07.785,10216,282,230,-190,1006,-117,10222.479151,1030.448931
469694,ags,15:37:07.790,10087,336,339,-68,1116,-208,10098.286290,1137.252830
469695,ags,15:37:07.795,10284,273,331,23,1134,-287,10292.946420,1169.980342


# 3.スケーリングデータに即したセンサデータの抽出
スケーリングデータに即したセンサデータの取得方法は以下の通りである．
1. センサデータ及びスケーリングデータの時刻列をdatetime化
2. timedelta関数を使用して，時刻計算を行い，実験実施日に合わせていく
3. センサデータの時刻列をインデックス化

  時刻列をインデックスにすることで，時間を使ってのデータのスライスが可能となる

4. スケーリングデータの時刻列を文字列型のリストにする
5. ４の操作を行うことでセンサデータからスケーリングデータに即したデータを抽出するのが容易になる

In [281]:
#センサデータの時刻合わせ
def datetime_order(sensor_data):
    sensor_data['時刻'] = pd.to_datetime(sensor_data['時刻'], format='%H:%M:%S.%f')
    sensor_data['時刻'] = sensor_data['時刻'] + dt.timedelta(days=44144)

#datetime_order(data1_a)
# datetime_order(data1_b)
# datetime_order(data1_c)
# datetime_order(data1_d)

datetime_order(data2_a)
# datetime_order(data2_b)
# datetime_order(data2_c)
# datetime_order(data2_d)

In [282]:
data2_a

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,2020-11-11 14:57:59.245,9801,898,-258,340,1,913,9845.433916,974.253560
1,ags,2020-11-11 14:57:59.250,9911,592,-290,291,44,559,9932.899124,631.742036
2,ags,2020-11-11 14:57:59.255,9996,431,-268,218,-188,279,10008.876111,400.885270
3,ags,2020-11-11 14:57:59.260,10150,453,-280,114,-84,-75,10163.961285,160.240444
4,ags,2020-11-11 14:57:59.265,10292,263,-122,-43,-267,-415,10296.082605,495.341296
...,...,...,...,...,...,...,...,...,...,...
469692,ags,2020-11-11 15:37:07.780,10184,356,313,-318,860,-20,10195.026287,917.128126
469693,ags,2020-11-11 15:37:07.785,10216,282,230,-190,1006,-117,10222.479151,1030.448931
469694,ags,2020-11-11 15:37:07.790,10087,336,339,-68,1116,-208,10098.286290,1137.252830
469695,ags,2020-11-11 15:37:07.795,10284,273,331,23,1134,-287,10292.946420,1169.980342


In [283]:
#data1_a_label_50['start_time'] = pd.to_datetime(data1_a_label_50['start_time'], format='%M:%S')
#data1_a_label_50['finish_time'] = pd.to_datetime(data1_a_label_50['finish_time'], format='%M:%S')

In [284]:
#data1_a_label_60['start_time'] = pd.to_datetime(data1_a_label_60['start_time'], format='%M:%S')
#data1_a_label_60['finish_time'] = pd.to_datetime(data1_a_label_60['finish_time'], format='%M:%S')

In [285]:
#data1_a_label_70['start_time'] = pd.to_datetime(data1_a_label_70['start_time'], format='%M:%S')
#data1_a_label_70['finish_time'] = pd.to_datetime(data1_a_label_70['finish_time'], format='%M:%S')

In [286]:
#data1_a_label_80['start_time'] = pd.to_datetime(data1_a_label_80['start_time'], format='%M:%S')
#data1_a_label_80['finish_time'] = pd.to_datetime(data1_a_label_80['finish_time'], format='%M:%S')

In [287]:
data2_a_label_50['start_time'] = pd.to_datetime(data2_a_label_50['start_time'], format='%M:%S')
data2_a_label_50['finish_time'] = pd.to_datetime(data2_a_label_50['finish_time'], format='%M:%S')

In [288]:
data2_a_label_60['start_time'] = pd.to_datetime(data2_a_label_60['start_time'], format='%M:%S')
data2_a_label_60['finish_time'] = pd.to_datetime(data2_a_label_60['finish_time'], format='%M:%S')

In [289]:
data2_a_label_70['start_time'] = pd.to_datetime(data2_a_label_70['start_time'], format='%M:%S')
data2_a_label_70['finish_time'] = pd.to_datetime(data2_a_label_70['finish_time'], format='%M:%S')

In [290]:
data2_a_label_80['start_time'] = pd.to_datetime(data2_a_label_80['start_time'], format='%M:%S')
data2_a_label_80['finish_time'] = pd.to_datetime(data2_a_label_80['finish_time'], format='%M:%S')

In [291]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_a)
display(data2_a_label_50)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,2020-11-11 14:57:59.245,9801,898,-258,340,1,913,9845.433916,974.253560
1,ags,2020-11-11 14:57:59.250,9911,592,-290,291,44,559,9932.899124,631.742036
2,ags,2020-11-11 14:57:59.255,9996,431,-268,218,-188,279,10008.876111,400.885270
3,ags,2020-11-11 14:57:59.260,10150,453,-280,114,-84,-75,10163.961285,160.240444
4,ags,2020-11-11 14:57:59.265,10292,263,-122,-43,-267,-415,10296.082605,495.341296
...,...,...,...,...,...,...,...,...,...,...
469692,ags,2020-11-11 15:37:07.780,10184,356,313,-318,860,-20,10195.026287,917.128126
469693,ags,2020-11-11 15:37:07.785,10216,282,230,-190,1006,-117,10222.479151,1030.448931
469694,ags,2020-11-11 15:37:07.790,10087,336,339,-68,1116,-208,10098.286290,1137.252830
469695,ags,2020-11-11 15:37:07.795,10284,273,331,23,1134,-287,10292.946420,1169.980342


,start_time,finish_time,emotion_level
0,1900-01-01 00:03:26,1900-01-01 00:03:30,0
1,1900-01-01 00:03:27,1900-01-01 00:03:31,0
2,1900-01-01 00:03:39,1900-01-01 00:03:43,-2
3,1900-01-01 00:03:40,1900-01-01 00:03:44,-2
4,1900-01-01 00:03:41,1900-01-01 00:03:45,-2
...,...,...,...
540,1900-01-01 00:18:06,1900-01-01 00:18:10,-2
541,1900-01-01 00:18:07,1900-01-01 00:18:11,-2
542,1900-01-01 00:18:08,1900-01-01 00:18:12,-2
543,1900-01-01 00:18:09,1900-01-01 00:18:13,-2


### 補足(実験実施時間)

本実験は，zoomレコーディング動画の経過時間でスケーリングデータのemotion_levelをつけたため動画の開始時刻を使って実際の実験実施時間と合わせる必要がある．

方法としては，経過時間と動画の開始時刻を足して実験実施時間を算出する．

動画の開始時間↓

第1セッション12時34分30秒開始

第2セッション15時11分44秒開始

In [292]:
#data1_a_label_50['start_time'] = data1_a_label_50['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
#data1_a_label_50['finish_time'] = data1_a_label_50['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [293]:
#data1_a_label_60['start_time'] = data1_a_label_60['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
#data1_a_label_60['finish_time'] = data1_a_label_60['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [294]:
#data1_a_label_70['start_time'] = data1_a_label_70['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
#data1_a_label_70['finish_time'] = data1_a_label_70['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [295]:
#data1_a_label_80['start_time'] = data1_a_label_80['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
#data1_a_label_80['finish_time'] = data1_a_label_80['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [296]:
data2_a_label_50['start_time'] = data2_a_label_50['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_a_label_50['finish_time'] = data2_a_label_50['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [297]:
data2_a_label_60['start_time'] = data2_a_label_60['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_a_label_60['finish_time'] = data2_a_label_60['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [298]:
data2_a_label_70['start_time'] = data2_a_label_70['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_a_label_70['finish_time'] = data2_a_label_70['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [299]:
data2_a_label_80['start_time'] = data2_a_label_80['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_a_label_80['finish_time'] = data2_a_label_80['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [300]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_a_label_50)

,start_time,finish_time,emotion_level
0,2020-11-11 15:15:10,2020-11-11 15:15:14,0
1,2020-11-11 15:15:11,2020-11-11 15:15:15,0
2,2020-11-11 15:15:23,2020-11-11 15:15:27,-2
3,2020-11-11 15:15:24,2020-11-11 15:15:28,-2
4,2020-11-11 15:15:25,2020-11-11 15:15:29,-2
...,...,...,...
540,2020-11-11 15:29:50,2020-11-11 15:29:54,-2
541,2020-11-11 15:29:51,2020-11-11 15:29:55,-2
542,2020-11-11 15:29:52,2020-11-11 15:29:56,-2
543,2020-11-11 15:29:53,2020-11-11 15:29:57,-2


In [301]:
#datetime化したラベルデータを保存
# data1_a_label_50.to_csv('emotion_label/label_datetime/emotion_A_20201111_1_datetime.csv', index=False)
# data1_a_label_60.to_csv('emotion_label/label_datetime/emotion_B_20201111_1_datetime.csv', index=False)
# data1_a_label_70.to_csv('emotion_label/label_datetime/emotion_C_20201111_1_datetime.csv', index=False)
# data1_a_label_80.to_csv('emotion_label/label_datetime/emotion_D_20201111_1_datetime.csv', index=False)

# data2_a_label_50.to_csv('emotion_label/label_datetime/emotion_A_20201111_2_datetime.csv', index=False)
# data2_a_label_60.to_csv('emotion_label/label_datetime/emotion_B_20201111_2_datetime.csv', index=False)
# data2_a_label_70.to_csv('emotion_label/label_datetime/emotion_C_20201111_2_datetime.csv', index=False)
# data2_a_label_80.to_csv('emotion_label/label_datetime/emotion_D_20201111_2_datetime.csv', index=False)

In [302]:
#datetime化した加速度及び角速度データを保存
# data1_a.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-121953632-ags_datetime.csv', index=False)
# data1_b.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-122231707-ags_datetime.csv', index=False)
# data1_c.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-121953885-ags_datetime.csv', index=False)
# data1_d.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-121954011-ags_datetime.csv', index=False)

# data2_a.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-145759193-ags_datetime.csv', index=False)
# data2_b.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-145733362-ags_datetime.csv', index=False)
# data2_c.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-150018208-ags_datetime.csv', index=False)
# data2_d.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-145806707-ags_datetime.csv', index=False)

In [303]:
#時刻をインデックスに
#data1_a.set_index("時刻", inplace=True)
# data1_b.set_index("時刻", inplace=True)
# data1_c.set_index("時刻", inplace=True)
# data1_d.set_index("時刻", inplace=True)

data2_a.set_index("時刻", inplace=True)
# data2_b.set_index("時刻", inplace=True)
# data2_c.set_index("時刻", inplace=True)
# data2_d.set_index("時刻", inplace=True)

In [304]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_a)

,種別,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
時刻,,,,,,,,,
2020-11-11 14:57:59.245,ags,9801,898,-258,340,1,913,9845.433916,974.253560
2020-11-11 14:57:59.250,ags,9911,592,-290,291,44,559,9932.899124,631.742036
2020-11-11 14:57:59.255,ags,9996,431,-268,218,-188,279,10008.876111,400.885270
2020-11-11 14:57:59.260,ags,10150,453,-280,114,-84,-75,10163.961285,160.240444
2020-11-11 14:57:59.265,ags,10292,263,-122,-43,-267,-415,10296.082605,495.341296
...,...,...,...,...,...,...,...,...,...
2020-11-11 15:37:07.780,ags,10184,356,313,-318,860,-20,10195.026287,917.128126
2020-11-11 15:37:07.785,ags,10216,282,230,-190,1006,-117,10222.479151,1030.448931
2020-11-11 15:37:07.790,ags,10087,336,339,-68,1116,-208,10098.286290,1137.252830


In [305]:
#data1_a_label_50["start_time"] = data1_a_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
#data1_a_label_50["finish_time"] = data1_a_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [306]:
#data1_a_label_60["start_time"] = data1_a_label_60["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
#data1_a_label_60["finish_time"] = data1_a_label_60["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [307]:
#data1_a_label_70["start_time"] = data1_a_label_70["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
#data1_a_label_70["finish_time"] = data1_a_label_70["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [308]:
#data1_a_label_80["start_time"] = data1_a_label_80["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
#data1_a_label_80["finish_time"] = data1_a_label_80["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [309]:
data2_a_label_50["start_time"] = data2_a_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_a_label_50["finish_time"] = data2_a_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [310]:
data2_a_label_60["start_time"] = data2_a_label_60["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_a_label_60["finish_time"] = data2_a_label_60["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [311]:
data2_a_label_70["start_time"] = data2_a_label_70["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_a_label_70["finish_time"] = data2_a_label_70["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [312]:
data2_a_label_80["start_time"] = data2_a_label_80["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_a_label_80["finish_time"] = data2_a_label_80["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [313]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_a_label_50)

,start_time,finish_time,emotion_level
0,2020-11-11 15:15:10,2020-11-11 15:15:14,0
1,2020-11-11 15:15:11,2020-11-11 15:15:15,0
2,2020-11-11 15:15:23,2020-11-11 15:15:27,-2
3,2020-11-11 15:15:24,2020-11-11 15:15:28,-2
4,2020-11-11 15:15:25,2020-11-11 15:15:29,-2
...,...,...,...
540,2020-11-11 15:29:50,2020-11-11 15:29:54,-2
541,2020-11-11 15:29:51,2020-11-11 15:29:55,-2
542,2020-11-11 15:29:52,2020-11-11 15:29:56,-2
543,2020-11-11 15:29:53,2020-11-11 15:29:57,-2


In [314]:
#startとfinishをリスト化
#data1_a_50_start_time = list(data1_a_label_50["start_time"])
#data1_a_50_finish_time = list(data1_a_label_50["finish_time"])

#data1_a_60_start_time = list(data1_a_label_60["start_time"])
#data1_a_60_finish_time = list(data1_a_label_60["finish_time"])

#data1_a_70_start_time = list(data1_a_label_70["start_time"])
#data1_a_70_finish_time = list(data1_a_label_70["finish_time"])

#data1_a_80_start_time = list(data1_a_label_80["start_time"])
#data1_a_80_finish_time = list(data1_a_label_80["finish_time"])

data2_a_50_start_time = list(data2_a_label_50["start_time"])
data2_a_50_finish_time = list(data2_a_label_50["finish_time"])

data2_a_60_start_time = list(data2_a_label_60["start_time"])
data2_a_60_finish_time = list(data2_a_label_60["finish_time"])

data2_a_70_start_time = list(data2_a_label_70["start_time"])
data2_a_70_finish_time = list(data2_a_label_70["finish_time"])

data2_a_80_start_time = list(data2_a_label_80["start_time"])
data2_a_80_finish_time = list(data2_a_label_80["finish_time"])

In [315]:
def sensor_segmentation_order(data, data_start_time, data_finish_time, data_label):
    a = [data[data_start_time[i]:data_finish_time[i]]for i in range(0, len(data_label), 1)]
    return a

#data1_a_record_1 = sensor_segmentation_order(data1_a, data1_a_50_start_time, data1_a_50_finish_time, data1_a_label_50)
#data1_a_record_2 = sensor_segmentation_order(data1_a, data1_a_60_start_time, data1_a_60_finish_time, data1_a_label_60)
#data1_a_record_3 = sensor_segmentation_order(data1_a, data1_a_70_start_time, data1_a_70_finish_time, data1_a_label_70)
#data1_a_record_4 = sensor_segmentation_order(data1_a, data1_a_80_start_time, data1_a_80_finish_time, data1_a_label_80)

data2_a_record_1 = sensor_segmentation_order(data2_a, data2_a_50_start_time, data2_a_50_finish_time, data2_a_label_50)
data2_a_record_2 = sensor_segmentation_order(data2_a, data2_a_60_start_time, data2_a_60_finish_time, data2_a_label_60)
data2_a_record_3 = sensor_segmentation_order(data2_a, data2_a_70_start_time, data2_a_70_finish_time, data2_a_label_70)
data2_a_record_4 = sensor_segmentation_order(data2_a, data2_a_80_start_time, data2_a_80_finish_time, data2_a_label_80)

# 4.特徴量算出
感情の識別を行うため学習器作成のために特徴量の算出を行う．今回特徴量としたのは，加速度の各軸及び角速度の各軸における時間領域での平均値，分散値，最大値，最小値及び，周波数領域での特徴量である最大ピーク周波数，最大ピーク値である．プログラムの処理手順を以下に示す．

1. 平均値の算出
2. 分散値の算出
3. 最大値の算出
4. 最小値の算出
5. 最大ピーク周波数及び最大ピーク値の算出
5. 全特徴量の数値を表記したデータフレームを作成

## 4.1 平均値算出
### 4.1.1 加速度

In [316]:
#加速度x,y,z,norm平均値
def gyr_mean_order(data_record, data_label):

    e = [np.mean(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.mean(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.mean(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.mean(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_mean_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_mean_list.columns = ["gyrx_mean", "gyry_mean", "gyrz_mean"]
#     gyr_mean_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_mean_list.columns = ["gyrx_mean", "gyry_mean", "gyrz_mean", "gyrn_mean"]
    mean_list = pd.concat([gyr_mean_list], axis=1, ignore_index=False)
    return mean_list

#data1_a_50_record_mean = gyr_mean_order(data1_a_record_1, data1_a_label_50)
#data1_a_60_record_mean = gyr_mean_order(data1_a_record_2, data1_a_label_60)
#data1_a_70_record_mean = gyr_mean_order(data1_a_record_3, data1_a_label_70)
#data1_a_80_record_mean = gyr_mean_order(data1_a_record_4, data1_a_label_80)

data2_a_50_record_mean = gyr_mean_order(data2_a_record_1, data2_a_label_50)
data2_a_60_record_mean = gyr_mean_order(data2_a_record_2, data2_a_label_60)
data2_a_70_record_mean = gyr_mean_order(data2_a_record_3, data2_a_label_70)
data2_a_80_record_mean = gyr_mean_order(data2_a_record_4, data2_a_label_80)

# a = data1_a_50_record_mean
# b = data1_a_60_record_mean
# c = data1_a_70_record_mean
# d = data1_a_80_record_mean

# e = data2_a_50_record_mean
# f = data2_a_60_record_mean
# g = data2_a_70_record_mean
# h = data2_a_80_record_mean

# mean_list = pd.concat([a, b, c, d], ignore_index=True)
# mean_list

In [317]:
#加速度x,y,z,norm分散値
def gyr_var_order(data_record, data_label):

    e = [np.var(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.var(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.var(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.var(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_var_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_var_list.columns = ["gyrx_var", "gyry_var", "gyrz_var"]
#     gyr_var_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_var_list.columns = ["gyrx_var", "gyry_var", "gyrz_var", "gyrn_var"]
    var_list = pd.concat([gyr_var_list], axis=1, ignore_index=False)
    return var_list

#data1_a_50_record_var = acc_gyr_var_order(data1_a_record_1, data1_a_label_50)
#data1_a_60_record_var = acc_gyr_var_order(data1_a_record_2, data1_a_label_60)
#data1_a_70_record_var = acc_gyr_var_order(data1_a_record_3, data1_a_label_70)
#data1_a_80_record_var = acc_gyr_var_order(data1_a_record_4, data1_a_label_80)

data2_a_50_record_var = gyr_var_order(data2_a_record_1, data2_a_label_50)
data2_a_60_record_var = gyr_var_order(data2_a_record_2, data2_a_label_60)
data2_a_70_record_var = gyr_var_order(data2_a_record_3, data2_a_label_70)
data2_a_80_record_var = gyr_var_order(data2_a_record_4, data2_a_label_80)

# a = data1_a_50_record_var
# b = data1_a_60_record_var
# c = data1_a_70_record_var
# d = data1_a_80_record_var

# e = data2_a_50_record_var
# f = data2_a_60_record_var
# g = data2_a_70_record_var
# h = data2_a_80_record_var

# var_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# var_list

In [318]:
#加速度x,y,z,norm分散値
def gyr_max_order(data_record, data_label):

    e = [np.max(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.max(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.max(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.max(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_max_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_max_list.columns = ["gyrx_max", "gyry_max", "gyrz_max"]
#     gyr_max_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_max_list.columns = ["gyrx_max", "gyry_max", "gyrz_max", "gyrn_max"]
    max_list = pd.concat([gyr_max_list], axis=1, ignore_index=False)
    return max_list

# data1_a_50_record_max = acc_gyr_max_order(data1_a_record_1, data1_a_label_50)
# data1_a_60_record_max = acc_gyr_max_order(data1_a_record_2, data1_a_label_60)
# data1_a_70_record_max = acc_gyr_max_order(data1_a_record_3, data1_a_label_70)
# data1_a_80_record_max = acc_gyr_max_order(data1_a_record_4, data1_a_label_80)

data2_a_50_record_max = gyr_max_order(data2_a_record_1, data2_a_label_50)
data2_a_60_record_max = gyr_max_order(data2_a_record_2, data2_a_label_60)
data2_a_70_record_max = gyr_max_order(data2_a_record_3, data2_a_label_70)
data2_a_80_record_max = gyr_max_order(data2_a_record_4, data2_a_label_80)

# a = data1_a_50_record_max
# b = data1_a_60_record_max
# c = data1_a_70_record_max
# d = data1_a_80_record_max

# e = data2_a_50_record_max
# f = data2_a_60_record_max
# g = data2_a_70_record_max
# h = data2_a_80_record_max

# max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# max_list

In [319]:
#加速度x,y,z,norm分散値
def gyr_min_order(data_record, data_label):

    e = [np.min(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.min(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.min(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.min(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_min_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_min_list.columns = ["gyrx_min", "gyry_min", "gyrz_min"]
#     gyr_min_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_min_list.columns = ["gyrx_min", "gyry_min", "gyrz_min", "gyrn_min"]
    min_list = pd.concat([gyr_min_list], axis=1, ignore_index=False)
    return min_list

#data1_a_50_record_min = acc_gyr_min_order(data1_a_record_1, data1_a_label_50)
#data1_a_60_record_min = acc_gyr_min_order(data1_a_record_2, data1_a_label_60)
#data1_a_70_record_min = acc_gyr_min_order(data1_a_record_3, data1_a_label_70)
#data1_a_80_record_min = acc_gyr_min_order(data1_a_record_4, data1_a_label_80)

data2_a_50_record_min = gyr_min_order(data2_a_record_1, data2_a_label_50)
data2_a_60_record_min = gyr_min_order(data2_a_record_2, data2_a_label_60)
data2_a_70_record_min = gyr_min_order(data2_a_record_3, data2_a_label_70)
data2_a_80_record_min = gyr_min_order(data2_a_record_4, data2_a_label_80)

# a = data1_a_50_record_min
# b = data1_a_60_record_min
# c = data1_a_70_record_min
# d = data1_a_80_record_min

# e = data2_a_50_record_min
# f = data2_a_60_record_min
# g = data2_a_70_record_min
# h = data2_a_80_record_min

# min_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# min_list

## 4.5 最大ピーク周波数及び最大ピーク値

In [320]:
def fq_amp_max_order(data_record, data_label):
    #サンプル数とサンプリング周期及びカットする周波数を設定
    N = 512 # サンプル数
    dt = 0.005 # サンプリング周期(sec)
    fc = 100
    t = np.arange(0, N*dt, dt) # 時間軸
    
    f = [data_record[i]["合成角速度"].head(512) for i in range(0, len(data_label), 1)]

    # 高速フーリエ変換(FFT)
    F = np.fft.fft(f)
    F_abs = np.abs(F) # 複素数を絶対値に変換
    F_abs_amp = F_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    F_abs_amp[0] = F_abs_amp[0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)

    # 周波数軸のデータ作成
    fq = np.linspace(0, 1.0/dt, N) # 周波数軸　linspace(開始,終了,分割数)

    # フィルタリング①（周波数でカット）＊＊＊＊＊＊
    F2 = np.copy(F) # FFT結果コピー
    for i in range(0, len(data_record), 1):
        F2[i][(fq > fc)] = 0
    F2_abs = np.abs(F2) # FFTの複素数結果を絶対値に変換
    F2_abs_amp = F2_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    for i in range(0, len(data_record), 1):
        F2_abs_amp[i][0] = F2_abs_amp[i][0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)
    
    fq_amp = pd.DataFrame(fq)
    amp = [pd.DataFrame(F2_abs_amp[i])for i in range(0, len(data_label), 1)]
    a = [pd.concat([fq_amp, amp[i]], axis=1, ignore_index=True)for i in range(0, len(data_label), 1)]

    a = [pd.DataFrame(a[i]).drop(a[i].index[[0]])for i in range(0, len(data_label), 1)]
    a = [a[i][a[i][1] == np.max(a[i][1])]for i in range(0, len(data_label), 1)]
    data_record_gyrn_fq_amp_max = pd.concat(a, ignore_index=True)
    data_record_gyrn_fq_amp_max.columns = ["gyrn_max_fq", "gyrn_max_peak"]
    
    b = pd.concat([data_record_gyrn_fq_amp_max], axis=1, ignore_index=False)
    return b

# data1_a_50_record_fq_amp_max = fq_amp_max_order(data1_a_record_1, data1_a_label_50)
# data1_a_60_record_fq_amp_max = fq_amp_max_order(data1_a_record_2, data1_a_label_60)
# data1_a_70_record_fq_amp_max = fq_amp_max_order(data1_a_record_3, data1_a_label_70)
# data1_a_80_record_fq_amp_max = fq_amp_max_order(data1_a_record_4, data1_a_label_80)

data2_a_50_record_fq_amp_max = fq_amp_max_order(data2_a_record_1, data2_a_label_50)
data2_a_60_record_fq_amp_max = fq_amp_max_order(data2_a_record_2, data2_a_label_60)
data2_a_70_record_fq_amp_max = fq_amp_max_order(data2_a_record_3, data2_a_label_70)
data2_a_80_record_fq_amp_max = fq_amp_max_order(data2_a_record_4, data2_a_label_80)

# a = data1_a_50_record_fq_amp_max
# b = data1_a_60_record_fq_amp_max
# c = data1_a_70_record_fq_amp_max
# d = data1_a_80_record_fq_amp_max

# e = data2_a_50_record_fq_amp_max
# f = data2_a_60_record_fq_amp_max
# g = data2_a_70_record_fq_amp_max
# h = data2_a_80_record_fq_amp_max

# fq_amp_max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# fq_amp_max_list

In [321]:
#全特徴量の行列の導出
# data1_a_50_record_all_feature = pd.concat([data1_a_50_record_mean, data1_a_50_record_var, data1_a_50_record_max, data1_a_50_record_min, data1_a_50_record_fq_amp_max], axis=1, ignore_index=False)
# data1_a_60_record_all_feature = pd.concat([data1_a_60_record_mean, data1_a_60_record_var, data1_a_60_record_max, data1_a_60_record_min, data1_a_60_record_fq_amp_max], axis=1, ignore_index=False)
# data1_a_70_record_all_feature = pd.concat([data1_a_70_record_mean, data1_a_70_record_var, data1_a_70_record_max, data1_a_70_record_min, data1_a_70_record_fq_amp_max], axis=1, ignore_index=False)
# data1_a_80_record_all_feature = pd.concat([data1_a_80_record_mean, data1_a_80_record_var, data1_a_80_record_max, data1_a_80_record_min, data1_a_80_record_fq_amp_max], axis=1, ignore_index=False)

data2_a_50_record_all_feature = pd.concat([data2_a_50_record_mean, data2_a_50_record_var, data2_a_50_record_max, data2_a_50_record_min, data2_a_50_record_fq_amp_max], axis=1, ignore_index=False)
data2_a_60_record_all_feature = pd.concat([data2_a_60_record_mean, data2_a_60_record_var, data2_a_60_record_max, data2_a_60_record_min, data2_a_60_record_fq_amp_max], axis=1, ignore_index=False)
data2_a_70_record_all_feature = pd.concat([data2_a_70_record_mean, data2_a_70_record_var, data2_a_70_record_max, data2_a_70_record_min, data2_a_70_record_fq_amp_max], axis=1, ignore_index=False)
data2_a_80_record_all_feature = pd.concat([data2_a_80_record_mean, data2_a_80_record_var, data2_a_80_record_max, data2_a_80_record_min, data2_a_80_record_fq_amp_max], axis=1, ignore_index=False)

In [322]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
data2_a_60_record_all_feature

,gyrx_mean,gyry_mean,gyrz_mean,gyrx_var,gyry_var,gyrz_var,gyrx_max,gyry_max,gyrz_max,gyrx_min,gyry_min,gyrz_min,gyrn_max_fq,gyrn_max_peak
0,35.461,-82.771,-12.776,21523.986479,1.139537e+06,30675.821824,462,3622,931,-665,-7304,-659,2.348337,597.483373
1,41.576,104.271,-31.387,11048.174224,4.279419e+05,16277.233231,462,3622,456,-433,-1926,-659,0.391389,369.979335
2,21.091,22.898,-34.053,5899.962719,1.013523e+05,5713.680191,322,1336,145,-226,-962,-379,0.782779,144.348261
3,21.096,17.128,-34.625,4920.556784,9.510960e+04,5170.244375,279,1336,145,-177,-962,-379,0.782779,149.596783
4,15.447,31.227,-22.915,3979.821191,6.543193e+04,3392.123775,273,1336,145,-177,-767,-287,1.174168,114.267170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,22.956,42.522,-27.508,1728.512064,5.841189e+04,3055.035936,163,945,145,-104,-797,-226,1.174168,88.251329
499,21.695,54.193,-27.739,1915.049975,6.778889e+04,3271.440879,163,1244,267,-135,-797,-226,1.174168,65.250229
500,20.961,27.404,-31.993,2308.777479,2.019019e+05,5444.190951,206,1439,322,-196,-3547,-336,0.782779,75.221671
501,19.446,36.201,-28.432,2150.831084,1.951729e+05,5333.901376,206,1439,322,-196,-3547,-336,2.739726,38.596097


In [323]:
#各特徴量データの保存
# data1_a_50_record_all_feature.to_csv('feature/A/data1_a_50_record_all_feature.csv', index=False)
# data1_a_60_record_all_feature.to_csv('feature/A/data1_a_60_record_all_feature.csv', index=False)
# data1_a_70_record_all_feature.to_csv('feature/A/data1_a_70_record_all_feature.csv', index=False)
# data1_a_80_record_all_feature.to_csv('feature/A/data1_a_80_record_all_feature.csv', index=False)

data2_a_50_record_all_feature.to_csv('feature/A/data2_a_50_record_all_feature.csv', index=False)
data2_a_60_record_all_feature.to_csv('feature/A/data2_a_60_record_all_feature.csv', index=False)
data2_a_70_record_all_feature.to_csv('feature/A/data2_a_70_record_all_feature.csv', index=False)
data2_a_80_record_all_feature.to_csv('feature/A/data2_a_80_record_all_feature.csv', index=False)

In [324]:
data2_a_label_60

,start_time,finish_time,emotion_level
0,2020-11-11 15:15:10,2020-11-11 15:15:14,0
1,2020-11-11 15:15:11,2020-11-11 15:15:15,0
2,2020-11-11 15:15:24,2020-11-11 15:15:28,-2
3,2020-11-11 15:15:25,2020-11-11 15:15:29,-2
4,2020-11-11 15:15:26,2020-11-11 15:15:30,-2
...,...,...,...
498,2020-11-11 15:29:50,2020-11-11 15:29:54,-2
499,2020-11-11 15:29:51,2020-11-11 15:29:55,-2
500,2020-11-11 15:29:52,2020-11-11 15:29:56,-2
501,2020-11-11 15:29:53,2020-11-11 15:29:57,-2


In [325]:
# data1_a_label_50_time = data1_a_label_50.drop("emotion_level", axis=1)
# data1_a_label_60_time = data1_a_label_60.drop("emotion_level", axis=1)
# data1_a_label_70_time = data1_a_label_70.drop("emotion_level", axis=1)
# data1_a_label_80_time = data1_a_label_80.drop("emotion_level", axis=1)

data2_a_label_50_time = data2_a_label_50.drop("emotion_level", axis=1)
data2_a_label_60_time = data2_a_label_60.drop("emotion_level", axis=1)
data2_a_label_70_time = data2_a_label_70.drop("emotion_level", axis=1)
data2_a_label_80_time = data2_a_label_80.drop("emotion_level", axis=1)

In [326]:
data2_a_label_60_time

,start_time,finish_time
0,2020-11-11 15:15:10,2020-11-11 15:15:14
1,2020-11-11 15:15:11,2020-11-11 15:15:15
2,2020-11-11 15:15:24,2020-11-11 15:15:28
3,2020-11-11 15:15:25,2020-11-11 15:15:29
4,2020-11-11 15:15:26,2020-11-11 15:15:30
...,...,...
498,2020-11-11 15:29:50,2020-11-11 15:29:54
499,2020-11-11 15:29:51,2020-11-11 15:29:55
500,2020-11-11 15:29:52,2020-11-11 15:29:56
501,2020-11-11 15:29:53,2020-11-11 15:29:57


In [327]:
# data1_a_50_record_all_feature = pd.concat([data1_a_label_50_time, data1_a_50_record_all_feature], axis=1)
# data1_a_60_record_all_feature = pd.concat([data1_a_label_60_time, data1_a_60_record_all_feature], axis=1)
# data1_a_70_record_all_feature = pd.concat([data1_a_label_70_time, data1_a_70_record_all_feature], axis=1)
# data1_a_80_record_all_feature = pd.concat([data1_a_label_80_time, data1_a_80_record_all_feature], axis=1)

data2_a_50_record_all_feature = pd.concat([data2_a_label_50_time, data2_a_50_record_all_feature], axis=1)
data2_a_60_record_all_feature = pd.concat([data2_a_label_60_time, data2_a_60_record_all_feature], axis=1)
data2_a_70_record_all_feature = pd.concat([data2_a_label_70_time, data2_a_70_record_all_feature], axis=1)
data2_a_80_record_all_feature = pd.concat([data2_a_label_80_time, data2_a_80_record_all_feature], axis=1)

In [328]:
data2_a_60_record_all_feature

,start_time,finish_time,gyrx_mean,gyry_mean,gyrz_mean,gyrx_var,gyry_var,gyrz_var,gyrx_max,gyry_max,gyrz_max,gyrx_min,gyry_min,gyrz_min,gyrn_max_fq,gyrn_max_peak
0,2020-11-11 15:15:10,2020-11-11 15:15:14,35.461,-82.771,-12.776,21523.986479,1.139537e+06,30675.821824,462,3622,931,-665,-7304,-659,2.348337,597.483373
1,2020-11-11 15:15:11,2020-11-11 15:15:15,41.576,104.271,-31.387,11048.174224,4.279419e+05,16277.233231,462,3622,456,-433,-1926,-659,0.391389,369.979335
2,2020-11-11 15:15:24,2020-11-11 15:15:28,21.091,22.898,-34.053,5899.962719,1.013523e+05,5713.680191,322,1336,145,-226,-962,-379,0.782779,144.348261
3,2020-11-11 15:15:25,2020-11-11 15:15:29,21.096,17.128,-34.625,4920.556784,9.510960e+04,5170.244375,279,1336,145,-177,-962,-379,0.782779,149.596783
4,2020-11-11 15:15:26,2020-11-11 15:15:30,15.447,31.227,-22.915,3979.821191,6.543193e+04,3392.123775,273,1336,145,-177,-767,-287,1.174168,114.267170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,2020-11-11 15:29:50,2020-11-11 15:29:54,22.956,42.522,-27.508,1728.512064,5.841189e+04,3055.035936,163,945,145,-104,-797,-226,1.174168,88.251329
499,2020-11-11 15:29:51,2020-11-11 15:29:55,21.695,54.193,-27.739,1915.049975,6.778889e+04,3271.440879,163,1244,267,-135,-797,-226,1.174168,65.250229
500,2020-11-11 15:29:52,2020-11-11 15:29:56,20.961,27.404,-31.993,2308.777479,2.019019e+05,5444.190951,206,1439,322,-196,-3547,-336,0.782779,75.221671
501,2020-11-11 15:29:53,2020-11-11 15:29:57,19.446,36.201,-28.432,2150.831084,1.951729e+05,5333.901376,206,1439,322,-196,-3547,-336,2.739726,38.596097


In [329]:
# data1_a_50_record_all_feature.to_csv('feature/A/data1_a_50_record_all_feature.csv', index=False)
# data1_a_60_record_all_feature.to_csv('feature/A/data1_a_60_record_all_feature.csv', index=False)
# data1_a_70_record_all_feature.to_csv('feature/A/data1_a_70_record_all_feature.csv', index=False)
# data1_a_80_record_all_feature.to_csv('feature/A/data1_a_80_record_all_feature.csv', index=False)

data2_a_50_record_all_feature.to_csv('feature/A/data2_a_50_record_all_feature.csv', index=False)
data2_a_60_record_all_feature.to_csv('feature/A/data2_a_60_record_all_feature.csv', index=False)
data2_a_70_record_all_feature.to_csv('feature/A/data2_a_70_record_all_feature.csv', index=False)
data2_a_80_record_all_feature.to_csv('feature/A/data2_a_80_record_all_feature.csv', index=False)

In [330]:
data2_a_60_record_all_feature

,start_time,finish_time,gyrx_mean,gyry_mean,gyrz_mean,gyrx_var,gyry_var,gyrz_var,gyrx_max,gyry_max,gyrz_max,gyrx_min,gyry_min,gyrz_min,gyrn_max_fq,gyrn_max_peak
0,2020-11-11 15:15:10,2020-11-11 15:15:14,35.461,-82.771,-12.776,21523.986479,1.139537e+06,30675.821824,462,3622,931,-665,-7304,-659,2.348337,597.483373
1,2020-11-11 15:15:11,2020-11-11 15:15:15,41.576,104.271,-31.387,11048.174224,4.279419e+05,16277.233231,462,3622,456,-433,-1926,-659,0.391389,369.979335
2,2020-11-11 15:15:24,2020-11-11 15:15:28,21.091,22.898,-34.053,5899.962719,1.013523e+05,5713.680191,322,1336,145,-226,-962,-379,0.782779,144.348261
3,2020-11-11 15:15:25,2020-11-11 15:15:29,21.096,17.128,-34.625,4920.556784,9.510960e+04,5170.244375,279,1336,145,-177,-962,-379,0.782779,149.596783
4,2020-11-11 15:15:26,2020-11-11 15:15:30,15.447,31.227,-22.915,3979.821191,6.543193e+04,3392.123775,273,1336,145,-177,-767,-287,1.174168,114.267170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,2020-11-11 15:29:50,2020-11-11 15:29:54,22.956,42.522,-27.508,1728.512064,5.841189e+04,3055.035936,163,945,145,-104,-797,-226,1.174168,88.251329
499,2020-11-11 15:29:51,2020-11-11 15:29:55,21.695,54.193,-27.739,1915.049975,6.778889e+04,3271.440879,163,1244,267,-135,-797,-226,1.174168,65.250229
500,2020-11-11 15:29:52,2020-11-11 15:29:56,20.961,27.404,-31.993,2308.777479,2.019019e+05,5444.190951,206,1439,322,-196,-3547,-336,0.782779,75.221671
501,2020-11-11 15:29:53,2020-11-11 15:29:57,19.446,36.201,-28.432,2150.831084,1.951729e+05,5333.901376,206,1439,322,-196,-3547,-336,2.739726,38.596097
